# TIME SERIES ANALYSIS

Business Understanding

Problem Statement

Project Goal
Stakeholders
Key Metrics
Features

Hypothesis

### Business Questions
1. Is the train dataset complete (has all the required dates)?
2. hich dates have the lowest and highest sales for each year (excluding days the store was closed)?
3. Compare the sales for each month across the years and determine which month of which year had the highest sales.
4. Did the earthquake impact sales?
5. Are certain stores or groups of stores selling more products? (Cluster, city, state, type)
6. Are sales affected by promotions, oil prices and holidays?
7. What analysis can we get from the date and its extractable features?
8. Which product family and stores did the promotions affect.
9. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)
10. Does the payment of wages in the public sector on the 15th and last days of the month influence the store sales.

# Data Understanding

# Environment variables management
from dotenv import dotenv_values

# Database connection
import pyodbc
import MySQLdb
import mysql.connector
import pymysql


# Visualization

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, display_html


 

# Data handling and utilities
import pandas as pd
import numpy as np
import re
import calendar
import warnings
import os
import pickle
# import joblib

# Data fetching
import requests
import zipfile

# Suppressing warnings to avoid cluttering the output
warnings.filterwarnings("ignore")

### Machine Learning libraries
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier



from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score,roc_curve,auc
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.metrics import roc_curve, roc_auc_score, auc

# load environment variables from.env file into dictionary
environment_variables = dotenv_values(".env")
 
# get the values for the environment variables
server = environment_variables.get("server")
login = environment_variables.get("login")
password = environment_variables.get("password")
database = environment_variables.get("database")
 
# Create a database connection string using pyodbc
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={login};PWD={password}"
#Establish a connection to the database
try:
    connection = pyodbc.connect(connection_string)
    print("Connection successful:", connection_string)    
except Exception as e:
    print("Connection failed:", e)

# Define the SQL query to show specific tables in the database
db_query = """
        SELECT *
        FROM INFORMATION_SCHEMA.TABLES
        WHERE TABLE_SCHEMA = 'dbo'
        """
# Read data from the SQL query result into a DataFrame using the established database connection
schema_df = pd.read_sql(db_query, connection)
 
#  Check whether data has been retrieved successfully to confirm successful connection to database
try:
    schema_df = pd.read_sql(db_query, connection)    
    print("Data retrieved successfully")
    print()
    print(schema_df)    
except Exception as e:
    print("Failed to retrieve data:", e)

# Define the SQL query to show specific tables in the database
db_query = """
        SELECT *
        FROM stores        
        """
# Read data from the SQL query result into a DataFrame using the established database connection
df_stores = pd.read_sql(db_query, connection)
 
# Display the DataFrame
df_stores.head()

# Define the SQL query to show specific tables in the database
db_query = """
        SELECT *
        FROM oil        
        """
# Read data from the SQL query result into a DataFrame using the established database connection
df_oil = pd.read_sql(db_query, connection)
 
# Display the DataFrame
df_oil.head()

# Define the SQL query to show specific tables in the database
db_query = """
        SELECT *
        FROM holidays_events        
        """
# Read data from the SQL query result into a DataFrame using the established database connection
df_holidays = pd.read_sql(db_query, connection)
 
# Display the DataFrame
df_holidays.head()

# URL of the file to download
url = "https://github.com/EfyaDufie2020/Career_Accelerator_LP3-Regression/raw/main/store-sales-forecasting.zip"
 
# Local file path where the file will be saved
local_file_path = '../Data/store-sales-forecasting.zip'
 
# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
 
# Send a GET request to the URL
response = requests.get(url)
 
# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Write the content of the response to the specified file path
    with open(local_file_path, "wb") as f:
        f.write(response.content)
    print("File downloaded successfully")
   
    # Extract the ZIP file
    with zipfile.ZipFile(local_file_path, 'r') as zip_ref:
        zip_ref.extractall(os.path.dirname(local_file_path))
    print("File extracted successfully")
else:
    print(f"Failed to download file. Status code: {response.status_code}")

# Read the downloaded CSV file into a DataFrame
df_train = pd.read_csv('../Data/train.csv')
 
# Display the DataFrame
df_train.head()

# Read the downloaded CSV file into a DataFrame
df_transactions = pd.read_csv('../Data/transactions.csv')
 
# Display the DataFrame
df_transactions.head()

# Read the downloaded CSV file into a DataFrame
df_test = pd.read_csv('../Data/test.csv')
 
# Display the DataFrame
df_test.head()

# Read the downloaded CSV file into a DataFrame
df_sample = pd.read_csv('../Data/sample_submission.csv')
 
# Display the DataFrame
df_sample.head()

# Merging the common columns ('store_nbr' and 'date') in the datasets using the inner merge() function
# Merge train_data with stores_df based on 'store_nbr' 
columnmerged_df1 = pd.merge(df_train,df_stores, on='store_nbr', how='left')
columnmerged_df1.head()

# Merge merged_df1 with trans_data based on 'date' and 'store_nbr' 
columnmerged_df2 = columnmerged_df1.merge(df_transactions, on=['date', 'store_nbr'], how='inner')
columnmerged_df2.head()

# Merge merged_df2 with holidays_events_df based on 'date' 
columnmerged_df3 = columnmerged_df2.merge(df_holidays, on='date', how='inner')
columnmerged_df3.head(3)

# Merge merged_df3 with oil_df based on 'date' 
columndata= columnmerged_df3.merge(df_oil, on='date', how='inner') 

columndata.head()


columndata.info()


# Merging the common columns ('store_nbr' and 'date') in the datasets using the inner merge() function
# Merge train_data with stores_df based on 'store_nbr' columnmerged_df1 = pd.merge(df_train,stores, on='store_nbr', how='left')
#merged_df1.head(3)
# Merge merged_df1 with trans_data based on 'date' and 'store_nbr' columnsmerged_df2 = merged_df1.merge(df_trx, on=['date', 'store_nbr'], how='inner')
# #merged_df2.head(3)
# Merge merged_df2 with holidays_events_df based on 'date' columnmerged_df3 = merged_df2.merge(holidays_events, on='date', how='inner')
#merged_df3.head(3)
 # Merge merged_df3 with oil_df based on 'date' columndata= merged_df3.merge(oil, on='date', how='inner')data.head(3)
 # View the first five rows of the merged dataset